# NHL Stats Preparation
## Collecting NHL Players' Stats from EliteProspects
This step gathers player performance data across leagues and seasons for downstream NHL prediction.

- Collect league-wide player stats for a given season from
https://eliteprospects.com/league/{league}/stats/{season}?page={n}

- Extract metadata, including individual player profile links.

- Build a unique player list from seasons 2000–2025 by storing profile links in a set.

- Scrape each player’s page to retrieve their first 5 years of NHL stats, filtered by seasons where they played more than 30 games.

- Gather pre-NHL stats for each player (junior, college, international, etc.).

### Import Libraries

In [1]:
import eliteprospects_scraper_api as ep
import pandas as pd
import os
import time
import random

### Collecting NHL Players' Stats from EliteProspects


In [2]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players from 2000-2025 season
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        print(f'Scraping {season}')
        nhl_players = ep.get_season_roster("nhl", season)
        nhl_players.to_csv(f'./data/nhl/players/nhl_players_{season}.csv', index=False, encoding='utf-8-sig')
        print(f'Finished scraping {season}')
else:
    print('NHL players metadata already exists. Skipping scraping.')

NHL players metadata already exists. Skipping scraping.


In [3]:
if not os.path.exists('./data/nhl/nhl_players_metadata.csv'):
    # Collect players' metadata from 2000-2025 season
    nhl_players_metadata = pd.DataFrame()
    for i in range(0, 25):
        season = f'20{str(i).zfill(2)}-20{str(i + 1).zfill(2)}'
        nhl_players = pd.read_csv(f'./data/nhl/players/nhl_players_{season}.csv')
        nhl_players_metadata = pd.concat([nhl_players_metadata, ep.get_players_metadata(nhl_players)]).reset_index(drop=True)
        # Remove duplicates
        nhl_players_metadata = nhl_players_metadata.drop_duplicates(subset=['playername']).reset_index(drop=True)
        print(f'Finished Merging Metadata for {season}')
else:
    print('NHL players metadata already exists. Skipping merging.')
    nhl_players_metadata = pd.read_csv('./data/nhl/nhl_players_metadata.csv')


NHL players metadata already exists. Skipping merging.


### Collect Stats for Each Player from 2000-2025 CSV file

In [15]:
# Define output file
output_path = './data/nhl/stats/nhl_players_stats.csv'
failed_path = './data/nhl/stats/failed_players.csv'

In [17]:
def get_players_stats_by_batch(players_metadata, start = 0, end = len(nhl_players_metadata)):
    end = min(end, len(players_metadata))
    print(f'Collecting stats for players {start} to {end}')

    for i in range(start, end):
        # Print divider for every 10 players
        if i % 10 == 0:
            print('----------------------------------------------------------------')

        player_metadata = players_metadata.iloc[i]
        player_name = player_metadata['playername']
        player_url = player_metadata['link']
        print(f"Collecting stats for {player_name} at {player_url}")

        try:
            player_stats = ep.get_player_stats(player_metadata)

            # Write to CSV file
            if os.path.exists(output_path):
                player_stats.to_csv(output_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                player_stats.to_csv(output_path, index=False, encoding='utf-8-sig')
            print(f'Successfully scraped stats for {player_name}')

            # Add random sleep to prevent getting blocked
            sleep_time = random.uniform(10, 180)
            print(f"Sleep for {sleep_time / 60:.2f} minutes to prevent getting blocked")
            time.sleep(sleep_time)
        except Exception as e:
            print(f"Failed to get stats for {player_name}: {e}")

            # Write failed player to CSV file
            if os.path.exists(failed_path):
                player_metadata.to_csv(failed_path, mode='a', header=False, index=False, encoding='utf-8-sig')
            else:
                player_metadata.to_csv(failed_path, index=False, encoding='utf-8-sig')

            print(f'Failed to scrape stats for {player_name}. Logged to {failed_path}')

### Fetch Players' Stats by Batch

### Fetch failed players

In [13]:
# Get unique players from nhl_players_stats.csv
if os.path.exists(output_path):
    nhl_players_stats = pd.read_csv(output_path)
    unique_players = set(nhl_players_stats['playername'].unique())
else:
    unique_players = set()

In [14]:
# Find players in nhl_players_metadata 0-200 not in unique_players
subset = nhl_players_metadata[0:200]
players_to_scrape = subset[~subset['playername'].isin(unique_players)].reset_index(drop=True)
players_to_scrape

,playername,fw_def,link


In [11]:
for i in range(len(players_to_scrape)):
    player_metadata = players_to_scrape.iloc[i]
    player_name = player_metadata['playername']
    player_url = player_metadata['link']
    print(f"Collecting stats for {player_name} at {player_url}")

    try:
        player_stats = ep.get_player_stats(player_metadata)

        # Write to CSV file
        if os.path.exists(output_path):
            player_stats.to_csv(output_path, mode='a', header=False, index=False, encoding='utf-8-sig')
        else:
            player_stats.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f'Successfully scraped stats for {player_name}')

        # Add random sleep to prevent getting blocked
        sleep_time = random.uniform(10, 180)
        print(f"Sleep for {sleep_time / 60:.2f} minutes to prevent getting blocked")
        time.sleep(sleep_time)
    except Exception as e:
        print(f"Failed to get stats for {player_name}: {e}")

        # Sleep for 1-3 minutes before trying the next player
        print(f"Sleeping for 1-3 minutes before trying the next player")
        time.sleep(random.uniform(60, 180))


Successfully scraped stats for Gary Suter
Sleep for 0.88 minutes to prevent getting blocked

Successfully scraped stats for Steve Thomas
Sleep for 0.61 minutes to prevent getting blocked

Successfully scraped stats for Jiri Slegr
Sleep for 2.59 minutes to prevent getting blocked

Successfully scraped stats for Phil Housley
Sleep for 0.19 minutes to prevent getting blocked

Successfully scraped stats for Dave Andreychuk
Sleep for 2.53 minutes to prevent getting blocked

Successfully scraped stats for Magnus Arvedson
Sleep for 1.45 minutes to prevent getting blocked

Successfully scraped stats for Trent Klatt
Sleep for 2.90 minutes to prevent getting blocked

Successfully scraped stats for Joé Juneau
Sleep for 2.20 minutes to prevent getting blocked
